In [47]:
from openai import OpenAI
import os
import base64
import requests
import pandas

from dotenv import load_dotenv
# Load API key
_ = load_dotenv()

In [48]:
client = OpenAI()

In [49]:
from pathlib import Path
                               
BASE = Path.cwd()                
BASE.mkdir(parents=True, exist_ok=True)

def save_df(df, name):
    path = BASE / name
    df.to_csv(path, index=False)
    print("✅  Saved →", path)

In [50]:
catalog = [
    "Eco Footwear", "Organic Apparel", "Digital Tablets",
    "Stylus Accessories", "Ergonomic Furniture", "Wellness Gadgets",
    "Smart Home", "Gaming Peripherals", "Travel Gear", "Pet Tech"
]

offer_library = [f"OFFER{str(i).zfill(2)}" for i in range(1,11)]

In [44]:
import json

def gpt_user_capsule():
    prompt = (
      "Create one fictitious LinkedIn headline (role + industry) and three distinct liked tweets. "
      "Also invent a 3‑step e‑commerce journey log (event_1, event_2, event_3), "
      "a list of three dwell times (floats), and three sku IDs viewed. "
      "Finally, pick TWO product categories that would interest this user "
      f"from the catalog {catalog} and the single best offer ID from {offer_library}. "
      "Return ONLY valid JSON with keys: linkedin_headline, liked_tweets, "
      "event_1, event_2, event_3, dwell_times, sku_views, cat_gt_1, cat_gt_2, best_offer_id."
    )
    resp = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role":"user","content":prompt}],
        temperature=0.7,
        response_format={"type":"json_object"}  
    )
    # Parse the JSON string into a dictionary
    return json.loads(resp.choices[0].message.content)

rows = []
for uid in range(1, 26):
    data = gpt_user_capsule()  # Now returns a dictionary
    data["user_id"] = uid
    rows.append(data)
personalization_df = pd.json_normalize(rows)
save_df(personalization_df, "personalization_data.csv")

✅  Saved → /Commjhub/jupyterhub/home/jaasminew/comm4190_S25/comm4190_S25_Research_Paper/personalization_data.csv


In [46]:
prompt_tmpl = (
  "Create ONE campaign brief with keys: theme, value_prop (≤12 words), target_audience (≤8 words). "
  "Also invent FIVE concise spec bullets and SIX SEO keywords (single or 2‑word phrases). "
  "Return ONLY valid JSON with those fields."
)

# Create a function similar to your gpt_user_capsule function
def gpt_campaign_brief():
    resp = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role":"user","content":prompt_tmpl}],
        temperature=1.1,
        response_format={"type":"json_object"}  
    )
    # Parse the JSON string into a dictionary
    return json.loads(resp.choices[0].message.content)

# Generate the data
rows = []
for cid in range(1, 11):
    data = gpt_campaign_brief()
    data["campaign_id"] = cid
    rows.append(data)

# Create DataFrame and save
content_df = pd.json_normalize(rows)
save_df(content_df, "content_data.csv")

✅  Saved → /Commjhub/jupyterhub/home/jaasminew/comm4190_S25/comm4190_S25_Research_Paper/content_data.csv
